# **Week 3:** Statistical Inference & Model Planning
#### July 5, 2023
---------------- 

This notebook is an exercise in exploratory data analysis, statistical testing, and model preparation using the Ames Iowa housing data (`ames_housing.csv`). 

[Link to Kaggle Dataset](https://www.kaggle.com/datasets/marcopale/housing)

Install necessary Python packages: Pandas, Matplotlib, Seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sma
import pylab as py
import statsmodels as sm
import scipy.stats as stats
import statsmodels.formula.api as smf

Read the `ames_housing.csv` CSV file into a Pandas Dataframe, and call it `df`. Output the top 5 rows of the dataframe. You can also open this file in Excel to better visualize the data.

In [ ]:
df = pd.read_csv('ames_housing.csv')
df.head()

What are the columns names in the dataset?

In [ ]:
df.columns

## Univariate EDA

Let's begin by exploring one of our nominal variables `Heating_QC` which categorizes the quality and condition of a home's heating system. Build a bar chat to show the frequencies of each condition (`Poor, Fair, Typical, Good, Excellent`) in the dataset.

In [ ]:
ax = sns.countplot(data=df, x='Heating_QC', order=['Poor', 'Fair', 'Typical', 'Good', 'Excellent'])
ax.set_title('Bar Graph of Heating System')
ax.set_xlabel('Heating System Quality')
ax.set_ylabel('Frequency')
plt.show()

We can also get the same information in tabular form by grouping the dataset on the `Heating_QC` variable.

In [ ]:
df.groupby('Heating_QC').size()

It's a good thing that very few houses have heating systems in `Poor` condition! *It will likely make sense to combine the categories for `Poor` and `Fair` for our later analysis.*

Now, let's explore one of our interval variables `Sale_Price` which provides the sale price for each home. Build a histogram for to show the distribution of this variable *in thousands of dollars.*

In [ ]:
ax = sns.histplot(x=df['Sale_Price']/1000)
ax.set_title('Histogram of Sale Price in Thousands of Dollars')
ax.set_xlabel('Sale Price (Thousands of $)')
ax.set_ylabel('Frequency')
plt.show()

From this histogram, we can conclude that most houses sell for less than $200,000, and there are a good number of outliers that are super expensive.

We can now use statistics to describe the *location, spread, and shape* of the data.

### Location

Compute the mean, median, and mode (and how often the mode occurs) of the sale price for homes in the dataset.

In [ ]:
np.mean(df['Sale_Price'])

In [ ]:
np.median(df['Sale_Price'])

In [ ]:
vals, counts = np.unique(df['Sale_Price'], return_counts=True)
mode_value = np.argwhere(counts == np.max(counts))
vals[mode_value].flatten().tolist()

In [ ]:
np.max(counts) # number of times the mode occurs in the dataset

Let's show two different distributions of sale price, one for each level of the binary variable `Central_Air.` Observe the difference in *location* based on the distributions. Add a Kernel Density Estimator to further differentiate the distributions.

In [ ]:
sns.histplot(data=df, x=df['Sale_Price']/1000, hue='Central_Air', kde=True)
plt.show()

We can immediately see that there are many more houses that have central air than do not in this dataset. The two distributions have different locations, with the blue distribution centered at a larger sale price than the orange. 

### Spread

Compute the range, IQR, sample variance, and standard deviation of the sale price for homes in the dataset.

In [ ]:
np.max(df['Sale_Price']) - np.min(df['Sale_Price'])

In [ ]:
q75, q25 = np.percentile(df['Sale_Price'], [75, 25])
iqr = q75 - q25
iqr

In [ ]:
sns.boxplot(data=df, y=df['Sale_Price']/1000)
plt.show()

In [ ]:
np.var(df['Sale_Price'], ddof = 1)

# ddof = “Delta Degrees of Freedom”
# This is the divisor used in the calculation for variance.
# By default, ddof is 0. We need to set it to 1 to fit the variance of the sample, rather than the population.

In [ ]:
stdev = np.std(df['Sale_Price'], ddof = 1)
stdev

Remember the standard deviation is the square root of the variance. We can verify this math by squaring what we get for the standard deviation to make sure it matches our variance.

In [ ]:
stdev**2

You can output most of these statistics automatically with the `describe()` function in Pandas.

In [ ]:
df['Sale_Price'].describe()

### Shape

Build a QQ plot to determine if sale price follows a normal distribution. 

In [ ]:
sma.qqplot(df['Sale_Price']/1000, line='45',fit=True)
py.show()

### Statistical Inference

Create a 95% confidence interval for the mean of sale price in our dataset.

In [ ]:
CI = stats.t.interval(confidence=0.95,
            df=len(df['Sale_Price'])-1,
            loc=np.mean(df['Sale_Price']), 
            scale=stats.sem(df['Sale_Price']))
CI

This can be interpreted as follows: "We expect that 95% of the time, this confidence interval will contain the population mean."

Let's conduct hypothesis testing (two-sided t-test for the mean) to test the null hypothesis that the mean sale price of homes in the dataset is $178,000. Let `alpha = 0.05`.

In [ ]:
stats.ttest_1samp(df['Sale_Price'], 178000, alternative='two-sided')

The p-value is greater than 0.05, so we cannot reject our null hypothesis. Instead, we can assume that the mean sale price of the population is not significantly different from $178,000.

## Bivariate EDA

### Continuous-Continuous Associations

To examine Continuous-Continuous associations, let's begin by exploring the relationship between sale price and greater living area (sq ft). Build a scatter plot between the two variables, with sale price as the target variable.

In [ ]:
ax = sns.relplot(data = df, y = df['Sale_Price']/1000, x = "Gr_Liv_Area")
ax.set(ylabel = 'Sales Price (Thousands $)',
       xlabel = 'Greater Living Area (Sqft)')
plt.show()

It looks like there is a relationship here, but we can't declare a statistical relationship without a formal hypothesis test.

### Continuous-Categorical Associations

To examine Continuous-Categorical associations, let's begin by exploring the relationship between external quality rating of the home and sale price. Build a bar chart between the two variables, showing the average sale price of homes with each value of exterior quality.

In [ ]:
ax = sns.catplot(data = df, y = "Sale_Price", x = "Exter_Qual", kind = "bar")
ax.set(ylabel = 'Sales Price (Thousands $)',
       xlabel = 'Exterior Quality')
plt.show()

Again, it looks like there is an association between the variables, but we can't rely soley on the graph. We need a hypothesis test.

Let's conduct a one-way ANOVA to determine whether there is a statistical difference in the mean of sale price across levels of exterior quality. 

In [ ]:
stats.f_oneway(df['Sale_Price'][df['Exter_Qual'] == 'Excellent'],
               df['Sale_Price'][df['Exter_Qual'] == 'Good'],
               df['Sale_Price'][df['Exter_Qual'] == 'Typical'],
               df['Sale_Price'][df['Exter_Qual'] == 'Fair'])

The p-value is so low, it's almost 0. We can conclude that there is a significant difference in sale price across different levels of exterior quality.

### Correlation

Conduct a Pearson correlation test to further explore the relationship between sale price and greater living area (sq ft).

In [ ]:
stats.pearsonr(df['Gr_Liv_Area'], df['Sale_Price'])

The correlation coefficient is 0.71. The p-value is so low, it's almost 0. Our null hypothesis here is that there is no linear relationship between the two variables; so, this means we can conclude there is a significant positive linear relationship between them.

### Simple Linear Regression

In [ ]:
model_slr = smf.ols("Sale_Price ~ Gr_Liv_Area", data = df).fit()
model_slr.summary()

The coefficient for `Gr_Liv_Area` is $111.69. The p-value is so low, it's almost 0. Since the coefficient is statistically significant, this means that we expect the price of a home to increase by $111.69 for every additional square foot of living space. 